# Project 1: Mining information from Text Data 
<hr>

## Task 2: Mining information from Text Data 

Using the whole anthologies abstract dataset. Extract a list of the authors and editors per publication and create baskets and perform a search of similar items, for example:

- basket 1: Mostafazadeh Davani Aida,Kiela Douwe,Lambert Mathias,Vidgen, Bertie Prabhakaran Vinodkumar, Waseem, Zeerak
- basket 2: Singh Sumer, Li Sheng

1. Find the frequent pair of items (2-tuples) using the naïve, A-priori and PCY algorithms. For each of these compare the time of execution and results for supports s=10, 50, 100. Comment your results. 

2. For the PCY algorithm, create up to 5 compact hash tables. What is  the difference in results and time of execution for 1,2,3,4 and 5 tables? Comment your results.

3. Find the final list of k-frequent items (k-tuples) for k=3 and 4. Experiment a bit and describe the best value for the support in each case. *Warning*: You can use any of the three algorithms, but be careful, because the algorithm can take too long if you don't chose it properly (well, basically don't use the naïve approach ;)).

4. Using one of the results of the previous items, for one k (k=2 or 3) find the possible clusters using the 1-NN criteria. Comment your results.

> 1-NN means that if you have a tuple {A,B,C} and {C,E,F} then because they share one element {C}, then they belong to the same cluster  {A,B,C,E,F}.

0. Import libraries

In [1]:
from urllib.request import urlopen
from io import BytesIO
from time import time

import pandas as pd
import gzip
import numpy as np
import itertools
import os

1. Download the data files

In [2]:
url  = 'https://aclanthology.org/anthology+abstracts.bib.gz'   # url where the file is stored
filename = 'anthology+abstracts.bib'                           # bib filename
folder   = 'data'                                              # folder name
minimum = 200                                                  # minimum number of words in the abtract to be considered 

# Create the path to store the files
if not os.path.exists(folder):
    os.makedirs(folder)

file = folder + '/' + filename

# Download the file if it doesn't exist locally
if(not os.path.exists(file)):
  print("Downloading " + url + " to /" + folder + "..." )
  with gzip.open(BytesIO(urlopen(url).read()), 'rb') as fb:
    with open(file, 'wb') as f:
        f.write(fb.read())
else:
  print("File " + filename + " already available in folder /" + folder)      

File anthology+abstracts.bib already available in folder /data


2. Load data

In [3]:
# Read authors / editors from the bib file

elements = []
with open(file, 'r',errors='ignore') as f:
    string = ''
    found = False
    # skip all lines until author/editor
    for line in f:
      if found:
        if '=' in line:                                        
          elements.append(string)
          string = ''
          found = False
        else:
          string = string + line
      if 'author = "' in line:                                 
        found = True
        string = string + line       
      if 'editor = "' in line:                                 
        found = True
        string = string + line        

3. Save list of authors/editors in a file

In [4]:
# Preporcess and clean the data and save it to the file ./data/authors.txt
authors_fname = './data/authors.txt'
baskets = []
for e in elements:
    new_string = e.replace("and", "")
    new_string = new_string.replace("\n", "")
    new_string = new_string.replace("    editor = ", "")
    new_string = new_string.replace("    author = ", "")
    new_string = new_string.replace(',', "")
    new_string = new_string.replace('"', "")
    new_string = new_string.replace('        ', ",")
    baskets.append(new_string)

with open(authors_fname, 'w') as file:    
    for s in baskets:        
        print(s, file=file)

nitems = 0
with open(authors_fname, "rt", encoding='latin1') as f:
    for line in f:
        C_k  = line.rstrip().split(',')
        nitems = nitems + len(C_k)

nbaskets = len(baskets)
print('Number of baskets:', nbaskets)
print('Number of items:', nitems)


Number of baskets: 70449
Number of items: 217901


<hr>

#### 1. Find the frequent pair of items (2-tuples) using the naïve, A-priori and PCY algorithms. For each of these compare the time of execution and results for supports s=10, 50, 100. Comment your results. 

# Naïve approach

In [5]:
# Read the date and generate the frozenset
def readdata(k, fname=authors_fname):    
    with open(fname, "rt", encoding='latin1') as f:
        for line in f:
            C_k  = line.rstrip().split(',')
            for itemset in itertools.combinations(C_k, k):
                    yield frozenset(itemset)  

In [6]:
nitems = 10
for C_k in readdata(k=2):
    print(C_k)    
    nitems -= 1
    if nitems == 0: 
        break    

frozenset({'Kiela Douwe', 'Mostafazadeh Davani Aida'})
frozenset({'Mostafazadeh Davani Aida', 'Lambert Mathias'})
frozenset({'Vidgen Bertie', 'Mostafazadeh Davani Aida'})
frozenset({'Prabhakaran Vinodkumar', 'Mostafazadeh Davani Aida'})
frozenset({'Waseem Zeerak', 'Mostafazadeh Davani Aida'})
frozenset({'Kiela Douwe', 'Lambert Mathias'})
frozenset({'Vidgen Bertie', 'Kiela Douwe'})
frozenset({'Prabhakaran Vinodkumar', 'Kiela Douwe'})
frozenset({'Waseem Zeerak', 'Kiela Douwe'})
frozenset({'Vidgen Bertie', 'Lambert Mathias'})


In [7]:
def get_C(k):
    start = time()
    C = {}
    for key in readdata(k):
        if key not in C:
            C[key] = 1
        else:
            C[key] += 1
    print("Took {}s for k={}".format((time() - start), k))
    return C

In [8]:
C1 = get_C(1)
print("C1 contains {} items".format(len(C1)))
C2 = get_C(2)
print("C2 contains {} items".format(len(C2)))

Took 0.3750131130218506s for k=1
C1 contains 61387 items
Took 0.6956984996795654s for k=2
C2 contains 247357 items


In [9]:
for (ck, n), _ in zip(C2.items(), range(5)):
    print(ck,n)

frozenset({'Kiela Douwe', 'Mostafazadeh Davani Aida'}) 2
frozenset({'Mostafazadeh Davani Aida', 'Lambert Mathias'}) 1
frozenset({'Vidgen Bertie', 'Mostafazadeh Davani Aida'}) 2
frozenset({'Prabhakaran Vinodkumar', 'Mostafazadeh Davani Aida'}) 3
frozenset({'Waseem Zeerak', 'Mostafazadeh Davani Aida'}) 2


In [10]:
def association_rules(nitems, C1, C2, L2):
    for i in range(len(L2)):
        A, B = L2[i]
        support_AB = C2[frozenset([A, B])]
        support_A = C1[frozenset([A])]
        conf_A_leads_to_B = support_AB / support_A

        support_B = C1[frozenset([B])]
        prob_B = support_B / nitems

        interest_A_leads_to_B = conf_A_leads_to_B - prob_B

        if interest_A_leads_to_B > 0.7:
            print("{} --> {} with interest {:3f}".format(A, B, interest_A_leads_to_B))

In [11]:
%time
supports = [10, 50, 100]
naive = []
L2s = []
for s in supports:
    t = time()
    L2 = {}
    for key, n in C2.items():
        if n >= s:
            L2[key] = n    

    L2 = [elem for elem in L2 if len(elem) > 1]  # clean our the list a bit.
    L2s.append(L2)
    t2 = round(time() - t,3)
    naive.append(str(len(L2)) + ' items with > ' + str(s) + ' occurrences in ' + str(t2) + 's')
    print("\nSupport: {}".format(s))
    association_rules(sum(1 for line in open(authors_fname)), C1, C2, L2)    

Wall time: 0 ns

Support: 10
Aguilar Gustavo --> Solorio Thamar with interest 0.907970
Rashid Ahmad --> Rezagholizadeh Mehdi with interest 0.999773
Negri Matteo --> Turchi Marco with interest 0.751200
Chaudhary Vishrav --> Guzm{\'a}n Francisco with interest 0.707709
Erdmann Grant --> Gwinnup Jeremy with interest 0.999702
Escolano Carlos --> Fonollosa Jos{\'e} A. R. with interest 0.832680
Wang Xing --> Tu Zhaopeng with interest 0.903896
Wei Daimeng --> Shang Hengchao with interest 0.908935
Lei Lizhi --> Wei Daimeng with interest 0.999844
Lei Lizhi --> Shang Hengchao with interest 0.999844
Guo Jiaxin --> Yang Hao with interest 0.999716
Wang Minghan --> Yang Hao with interest 0.937216
Lei Lizhi --> Yang Hao with interest 0.999716
Li Bei --> Zhu Jingbo with interest 0.832354
Xiao Tong --> Zhu Jingbo with interest 0.885384
Hangya Viktor --> Fraser Alexer with interest 0.840998
Stojanovski Dario --> Fraser Alexer with interest 0.921970
Crego Josep --> Senellart Jean with interest 0.949333
Ch

In [12]:
print(naive)

['1705 items with > 10 occurrences in 0.037s', '12 items with > 50 occurrences in 0.039s', '0 items with > 100 occurrences in 0.037s']


# A-priori algorithm

In [13]:
supports = [10, 50, 100]
apriori = []
L2s = []
for s in supports:

    print('Threshold = ', s)
    t = time()
    C1 = get_C(1)
    print("C1 contains {} items".format(len(C1)))
    
    # filter stage
    L1 = {}
    for key, count in C1.items():
        if count >= s:
            L1[key] = count
    
    print('{} of those items with > {} occurrences'.format(len(L1),s))

    C2_items = set([a.union(b) for a in L1.keys() for b in L1.keys()]) # List comprehensions in python
    
    # find frequent 2-tuples
    C2 = {}
    for key in readdata(k=2):
        # filter out non-frequent tuples
        if key not in C2_items:
            continue

        # record frequent tuples
        if key not in C2:
            C2[key] = 1
        else:
            C2[key] += 1
    
    print("C2 contains {} items".format(len(C2)))

    # filter stage
    L2 = {}    
    for key, count in C2.items():
        if count >= s:
            L2[key] = count        
  
    #print(L2)
    t2 = round(time() - t,3)
    print('A-priori: {} items with >{} occurrences\n'.format(len(L2), s))    
    apriori.append(str(len(L2)) + ' items with > ' + str(s) + ' occurrences in ' + str(t2) + 's')
    L2s.append(L2) 


Threshold =  10
Took 0.5669822692871094s for k=1
C1 contains 61387 items
4154 of those items with > 10 occurrences
C2 contains 37362 items
A-priori: 1705 items with >10 occurrences

Threshold =  50
Took 0.26999974250793457s for k=1
C1 contains 61387 items
450 of those items with > 50 occurrences
C2 contains 3506 items
A-priori: 12 items with >50 occurrences

Threshold =  100
Took 0.3229794502258301s for k=1
C1 contains 61387 items
95 of those items with > 100 occurrences
C2 contains 436 items
A-priori: 0 items with >100 occurrences



In [14]:
print(apriori)

['1705 items with > 10 occurrences in 52.205s', '12 items with > 50 occurrences in 3.75s', '0 items with > 100 occurrences in 0.622s']


### PCY algorithm

In [15]:
# Hash table
max_hash1 = 10 * 1000000
H1 = np.zeros((max_hash1, ), dtype=np.int64)

for key in readdata(k=2):
    hash_cell_1 = hash(key) % max_hash1
    H1[hash_cell_1] += 1

In [16]:
supports = [10, 50, 100]
pcy = []
L2s = []
for s in supports:

    print('Threshold = ', s)
    t = time()
    C1 = get_C(1)
    print("C1 contains {} items".format(len(C1)))
    
    L1 = {}
    for key, count in C1.items():
        if count >= s:
            L1[key] = count
    
    print('{} of those items with > {} occurrences'.format(len(L1),s))

    C2_items = set([a.union(b) for a in L1.keys() for b in L1.keys()]) # List comprehensions in python  

    # find frequent 2-tuples
    C2 = {}
    N = 10
    for key in readdata(k=2):
    
        # hash-based filtering stage from PCY
        hash_cell_1 = hash(key) % max_hash1
        if H1[hash_cell_1] < s:
            continue

        # filter out non-frequent tuples
        if key not in C2_items:
            continue

        # record frequent tuples
        if key not in C2:
            C2[key] = 1
        else:
            C2[key] += 1
        
    print("C2 contains {} items".format(len(C2)))

    # filter stage
    L2 = {}    
    for key, count in C2.items():
        if count >= s:
            L2[key] = count
            
    t2 = round(time() - t,3)
    print('PCY: {} items with >{} occurrences\n'.format(len(L2), n))
    pcy.append(str(len(L2)) + ' items with > ' + str(s) + ' occurrences in ' + str(t2) + 's')    
    L2s.append(L2) 

Threshold =  10
Took 0.26100802421569824s for k=1
C1 contains 61387 items
4154 of those items with > 10 occurrences
C2 contains 1741 items
PCY: 1705 items with >1 occurrences

Threshold =  50
Took 0.2680017948150635s for k=1
C1 contains 61387 items
450 of those items with > 50 occurrences
C2 contains 12 items
PCY: 12 items with >1 occurrences

Threshold =  100
Took 0.26900196075439453s for k=1
C1 contains 61387 items
95 of those items with > 100 occurrences
C2 contains 0 items
PCY: 0 items with >1 occurrences



In [17]:
print('Naïve:', naive)
print('A-priori:', apriori)
print('PCY:', pcy)

Naïve: ['1705 items with > 10 occurrences in 0.037s', '12 items with > 50 occurrences in 0.039s', '0 items with > 100 occurrences in 0.037s']
A-priori: ['1705 items with > 10 occurrences in 52.205s', '12 items with > 50 occurrences in 3.75s', '0 items with > 100 occurrences in 0.622s']
PCY: ['1705 items with > 10 occurrences in 43.831s', '12 items with > 50 occurrences in 4.043s', '0 items with > 100 occurrences in 0.987s']


As we can see the results obtained are the same with the 3 methods but the number of items to count in C2 is less with A-priori algorithm and even more with the PCY algorithm which makes those algorithms better in terms of computational memory requirements.

<hr>

#### 2. For the PCY algorithm, create up to 5 compact hash tables. What is  the difference in results and time of execution for 1,2,3,4 and 5 tables? Comment your results.

In [18]:
supports = [10, 50, 100]

# Definie hash tables
max_hash1 = 5*1000000-673
max_hash2 = 5*1000000+673
max_hash3 = 5*1000000-1673
max_hash4 = 5*1000000+1673
max_hash5 = 5*1000000+11673

for s in supports:

    t = time()
    print('Threshold = {}\n'.format(s))

    H1 = np.zeros((max_hash1,), dtype=int)
    H2 = np.zeros((max_hash2,), dtype=int)
    H3 = np.zeros((max_hash3,), dtype=int)
    H4 = np.zeros((max_hash4,), dtype=int)
    H5 = np.zeros((max_hash5,), dtype=int)

    for key in readdata(k=2):
        hash_cell_1 = hash(key) % max_hash1
        H1[hash_cell_1] += 1
        hash_cell_2 = hash(key) % max_hash2
        H2[hash_cell_2] += 1
        hash_cell_3 = hash(key) % max_hash3
        H3[hash_cell_3] += 1
        hash_cell_4 = hash(key) % max_hash4
        H4[hash_cell_4] += 1
        hash_cell_5 = hash(key) % max_hash5
        H5[hash_cell_5] += 1

    # compact hash table
    H_good_1 = set(np.where(H1 >= s)[0])
    H_good_2 = set(np.where(H2 >= s)[0])
    H_good_3 = set(np.where(H3 >= s)[0])
    H_good_4 = set(np.where(H4 >= s)[0])
    H_good_5 = set(np.where(H5 >= s)[0])

    del H1
    del H2
    del H3
    del H4
    del H5

    # find frequent 1-tuples (individual items)
    C1 = {}
    for key in readdata(k=1):
        if key not in C1:
            C1[key] = 1
        else:
            C1[key] += 1    

    # filter stage
    L1 = {}
    for key, count in C1.items():
        if count >= s:
            L1[key] = count

    C2_items = set([a.union(b) for a in L1.keys() for b in L1.keys()]) # List comprehensions in python

    # find frequent 2-tuples
    C2 = {}


    for i in range (1,6):
        for key in readdata(k=2):
            # hash-based filtering stage from PCY
            if i >= 1:
                hash_cell_1 = hash(key) % max_hash1
                if hash_cell_1 not in H_good_1:
                    continue
            if i >= 2:
                hash_cell_2 = hash(key) % max_hash2
                if hash_cell_2 not in H_good_2:
                    continue
            if i >= 3:
                hash_cell_3 = hash(key) % max_hash3
                if hash_cell_3 not in H_good_3:
                    continue
            if i >= 4:
                hash_cell_4 = hash(key) % max_hash4
                if hash_cell_4 not in H_good_4:
                    continue
            if i >= 5:
                hash_cell_5 = hash(key) % max_hash5
                if hash_cell_5 not in H_good_5:
                    continue
                    
            # filter out non-frequent tuples
            if key not in C2_items:
                continue

            # record frequent tuples
            if key not in C2:
                C2[key] = 1
            else:
                C2[key] += 1
        
        # filter stage
        L2 = {}
        for key, count in C2.items():
            if count >= s:
                L2[key] = count
        t2 = round(time() - t,3)

        print('{} items with > {} occurrences found in {} using {} hashing functions'.format(len(L2), s, t2, i))

Threshold = 10

1705 items with > 10 occurrences found in 39.009 using 1 hashing functions
1705 items with > 10 occurrences found in 39.489 using 2 hashing functions
1705 items with > 10 occurrences found in 39.97 using 3 hashing functions
1705 items with > 10 occurrences found in 40.475 using 4 hashing functions
1705 items with > 10 occurrences found in 40.99 using 5 hashing functions
Threshold = 50

12 items with > 50 occurrences found in 5.645 using 1 hashing functions
12 items with > 50 occurrences found in 6.042 using 2 hashing functions
12 items with > 50 occurrences found in 6.428 using 3 hashing functions
12 items with > 50 occurrences found in 6.812 using 4 hashing functions
12 items with > 50 occurrences found in 7.199 using 5 hashing functions
Threshold = 100

0 items with > 100 occurrences found in 2.642 using 1 hashing functions
0 items with > 100 occurrences found in 3.03 using 2 hashing functions
0 items with > 100 occurrences found in 3.415 using 3 hashing functions
0 i

We have seen that we do not improve the execution time by addig more hashing functions but we can appreciare that, by splitting memory among too many hash-tables, the hash-tables get smaller, resulting in more collisions and, too many collisions, may result in an unefficient filtering out of infrequent pairs.

<hr>

#### 3. Find the final list of k-frequent items (k-tuples) for k=3 and 4. Experiment a bit and describe the best value for the support in each case. *Warning*: You can use any of the three algorithms, but be careful, because the algorithm can take too long if you don't chose it properly (well, basically don't use the naïve approach ;)).

In [19]:
supports = [10, 20, 30]
k_tuples = [3, 4]

for s in supports:
    for k in k_tuples:

        t = time()

        # find frequent 1-tuples (individual items)
        C1 = {}
        for key in readdata(k=1):
            if key not in C1:
                C1[key] = 1
            else:
                C1[key] += 1    
            
        # filter stage
        L1 = {}
        for key, count in C1.items():
            if count >= s:
                L1[key] = count
            
        # find frequent 2-tuples    
        C2 = {}
        for key in readdata(k=2):
        # record frequent tuples
            if key not in C2:
                C2[key] = 1
            else:
                C2[key] += 1

        # filter stage
        L2 = {}
        for key, count in C2.items():
            if count >= s:
                L2[key] = count

        if k == 3:

            C3_items = set([a.union(b) for a in L2.keys() for b in L2.keys() ]) # List comprehensions in python

            # Hash table
            max_hash1 = 10 * 1000000
            H1 = np.zeros((max_hash1, ), dtype=int)

            for key in readdata(k=3):
                hash_cell_1 = hash(key) % max_hash1
                H1[hash_cell_1] += 1
    
            # find frequent 3-tuples
            C3 = {}
            for key in readdata(k=3):
                # hash-based filtering stage from PCY
                hash_cell_1 = hash(key) % max_hash1
                if H1[hash_cell_1] < s:
                    continue
    
                # filter out non-frequent tuples
                if key not in C3_items:
                    continue

                # record frequent tuples
                if key not in C3:
                    C3[key] = 1
                else:
                    C3[key] += 1

            # filter stage
            L3 = {}
            for key, count in C3.items():
                if count >= s:
                    L3[key] = count
            t2 = round(time() - t,3)

            print('{} {}-tuples items with > {} occurrences found in {}'.format(len(L3), k, s, t2))    

        else:
            # find frequent 3-tuples
            C3 = {}
            for key in readdata(k=3):            
                # record frequent tuples
                if key not in C3:
                    C3[key] = 1
                else:
                    C3[key] += 1

            # filter stage
            L3 = {}
            for key, count in C3.items():
                if count >= s:
                    L3[key] = count
    
            C4_items = set([a.union(b) for a in L3.keys() for b in L3.keys() ]) # List comprehensions in python  

            # hash table
            max_hash1 = 10 * 1000000
            H1 = np.zeros((max_hash1, ), dtype=int)

            for key in readdata(k=4):
                hash_cell_1 = hash(key) % max_hash1
                H1[hash_cell_1] += 1
            
            # find frequent 3-tuples
            C4 = {}

            for key in readdata(k=4):
                # hash-based filtering stage from PCY
                hash_cell_1 = hash(key) % max_hash1
                if H1[hash_cell_1] < s:
                    continue
            
                # filter out non-frequent tuples
                if key not in C4_items:
                    continue

                # record frequent tuples
                if key not in C4:
                    C4[key] = 1
                else:
                    C4[key] += 1

            # filter stage
            L4 = {}
            for key, count in C4.items():
                if count >= s:
                    L4[key] = count
            t2 = round(time() - t,3)    
            print('{} {}-tuples items with > {} occurrences found in {}\n'.format(len(L4), k, s, t2))      
                      
            # saving result for next exercise
            if s == 10:
                result = L4   

342 3-tuples items with > 10 occurrences found in 6.963
103 4-tuples items with > 10 occurrences found in 12.85

8 3-tuples items with > 20 occurrences found in 5.288
0 4-tuples items with > 20 occurrences found in 10.566

1 3-tuples items with > 30 occurrences found in 3.252
0 4-tuples items with > 30 occurrences found in 11.234



<hr>

#### 4. Using one of the results of the previous items, for one k (k=2 or 3) find the possible clusters using the 1-NN criteria. Comment your results.